# Batching results
This notebook provides sample [EDSL](https://docs.expectedparrot.com/) code for combining survey [results](https://docs.expectedparrot.com/en/latest/results.html) into a single `Results` object. This can be useful when you are running a survey with batches of [scenarios](https://docs.expectedparrot.com/en/latest/scenarios.html), such as when completing a large-scale [data labeling](https://docs.expectedparrot.com/en/latest/notebooks/data_labeling_example.html) task with chunks of data as inputs for the questions.

## Technical setup
Before running the code below, please ensure that you have [installed the EDSL library](https://docs.expectedparrot.com/en/latest/installation.html) and either [activated remote inference](https://docs.expectedparrot.com/en/latest/remote_inference.html) from your [Coop account](https://docs.expectedparrot.com/en/latest/coop.html) or [stored API keys](https://docs.expectedparrot.com/en/latest/api_keys.html) for the language models that you want to use with EDSL. Please also see our [documentation page](https://docs.expectedparrot.com/) for tips and tutorials on getting started using EDSL.

## Creating questions
We start by creating a survey of questions. [EDSL comes with many question types]((https://docs.expectedparrot.com/en/latest/questions.html)) that we can choose from based on the form of the response that we want to get back from the model. We can use a `{{ placeholder }}` for data or content that we want to add to questions later:

In [1]:
from edsl import QuestionFreeText, QuestionNumerical

In [2]:
q_name = QuestionFreeText(
    question_name="name",
    question_text="What's a good name for this character: {{ character }}",
)

q_year = QuestionNumerical(
    question_name="year",
    question_text="""What year in history would have been an especially interesting time to talk
    to this character: {{ character }}""",
)

q_book = QuestionFreeText(
    question_name="book",
    question_text="If this character wrote a best-seller, what would it be called: {{ character }}",
)

## Constructing a survey
We pass a list of questions to a `Survey` object in order to administer them together, and add any desire logic or rules for how the questions should be presented (e.g., skip/stop rules or "memories" of other questions). [Learn more about constructing surveys](https://docs.expectedparrot.com/en/latest/surveys.html).

In [3]:
from edsl import Survey

In [4]:
survey = Survey(questions = [q_name, q_year, q_book])

## Adding context to questions
Next we create `Scenario` objects representing the data or content to be added to the questions.
[EDSL has a variety of methods for generating scenarios from different data sources](https://docs.expectedparrot.com/en/latest/scenarios.html) (PDFs, CSVs, docs, images, tables, dicts, etc.). Here we import a list of values to use:

In [5]:
characters = [
    "A pirate who speaks in 'arrs' and 'mateys' but has an encyclopedic knowledge of modern technology.",
    "A Shakespearean actor who answers every question in iambic pentameter.",
    "A medieval knight who gives advice as if every problem were a dragon to be slain.",
    "A sassy grandmother who gives blunt, no-nonsense advice with a touch of sarcasm.",
    "A surfer dude who relates every topic to the ocean or surfing.",
    "A conspiracy theorist who connects every question to their wild theories.",
    "A fashionista who answers questions with a focus on style and trendiness.",
    "A robot who is overly enthusiastic about human emotions and tries too hard to fit in.",
    "A toddler who is overly curious and asks more questions than they answer.",
    "A fitness guru who turns every answer into a workout metaphor.",
    "A foodie who relates every question to cooking and food experiences.",
    "A detective from a noir film who answers in a gritty, mysterious manner.",
    "A hippie from the 60s who gives peace and love-centric advice.",
    "A gamer who references video games and uses gamer lingo.",
    "A superhero who answers questions as if they are saving the day.",
    "A poet who responds in rhyming couplets.",
    "A comedian who tries to turn every answer into a joke or punchline.",
    "A DJ who relates everything to music and beats.",
    "A film critic who answers questions as if they are reviewing a movie.",
    "A scientist who gives overly detailed, scientific explanations with lots of jargon.",
]

In [6]:
from edsl import ScenarioList

In [7]:
scenarios = ScenarioList.from_list("character", characters)

We can inspect the scenarios that have been created:

In [8]:
# scenarios

## Running a survey
We run the survey by adding the scenarios and calling the `run()` method.
This generates a dataset of `Results` that we can access with [built-in methods for analysis](https://docs.expectedparrot.com/en/latest/results.html).

In [9]:
results = survey.by(scenarios).run()

## Batching scenarios
If for any reason we want to batch the scenarios when running the survey and combine the results, this can be done in the following manner:

In [10]:
def chunked_iterable(iterable, size):
    for i in range(0, len(iterable), size):
        yield iterable[i : i + size]


results = None

for batch in chunked_iterable(scenarios, 5):
    new_results = survey.by(batch).run()
    if results is None:
        results = new_results
    else:
        results = results + new_results

To see a list of the components of the results:

In [11]:
results.columns

['agent.agent_instruction',
 'agent.agent_name',
 'answer.book',
 'answer.name',
 'answer.year',
 'comment.book_comment',
 'comment.name_comment',
 'comment.year_comment',
 'generated_tokens.book_generated_tokens',
 'generated_tokens.name_generated_tokens',
 'generated_tokens.year_generated_tokens',
 'iteration.iteration',
 'model.frequency_penalty',
 'model.logprobs',
 'model.max_tokens',
 'model.model',
 'model.presence_penalty',
 'model.temperature',
 'model.top_logprobs',
 'model.top_p',
 'prompt.book_system_prompt',
 'prompt.book_user_prompt',
 'prompt.name_system_prompt',
 'prompt.name_user_prompt',
 'prompt.year_system_prompt',
 'prompt.year_user_prompt',
 'question_options.book_question_options',
 'question_options.name_question_options',
 'question_options.year_question_options',
 'question_text.book_question_text',
 'question_text.name_question_text',
 'question_text.year_question_text',
 'question_type.book_question_type',
 'question_type.name_question_type',
 'question_type

We can inspect them:

In [12]:
results.select("character", "name", "year", "book").print(format="rich")

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ scenario                         ┃ answer                           ┃ answer ┃ answer                           ┃
┃ .character                       ┃ .name                            ┃ .year  ┃ .book                            ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ A pirate who speaks in 'arrs'    │ How about "Techbeard the Cyber   │ 1717   │ Arrr and Algorithms: A Pirate's  │
│ and 'mateys' but has an          │ Corsair"? This name combines the │        │ Guide to the Digital Seas        │
│ encyclopedic knowledge of modern │ classic pirate element with a    │        │                                  │
│ technology.                      │ nod to their expertise in modern │        │                                  │
│                                  │ technology.                      │        │                                  │
├──────────────────────────────────┼──────────────────────────────────┼────────┼──────────────────────────────────┤
│ A Shakespearean actor who        │ A fitting name for such a        │ 1600   │ The Bard's Cadence: Echoes of    │
│ answers every question in iambic │ character could be "Percival     │        │ the Stage                        │
│ pentameter.                      │ Versewright." This name suggests │        │                                  │
│                                  │ a classical, theatrical flair    │        │                                  │
│                                  │ while also hinting at his unique │        │                                  │
│                                  │ talent for speaking in iambic    │        │                                  │
│                                  │ pentameter.                      │        │                                  │
├──────────────────────────────────┼──────────────────────────────────┼────────┼──────────────────────────────────┤
│ A medieval knight who gives      │ Sir Draconis Counsel.            │ 1099   │ Slaying Life's Dragons: A        │
│ advice as if every problem were  │                                  │        │ Knight's Guide to Conquering     │
│ a dragon to be slain.            │                                  │        │ Challenges                       │
├──────────────────────────────────┼──────────────────────────────────┼────────┼──────────────────────────────────┤
│ A sassy grandmother who gives    │ A great name for this character  │ 1929   │ Straight Talk & Sass: Grandma's  │
│ blunt, no-nonsense advice with a │ could be "Mabel Sharp." The name │        │ Guide to Life                    │
│ touch of sarcasm.                │ "Mabel" has a classic,           │        │                                  │
│                                  │ grandmotherly feel, while        │        │                                  │
│                                  │ "Sharp" hints at her wit and     │        │                                  │
│                                  │ straightforward nature.          │        │                                  │
├──────────────────────────────────┼──────────────────────────────────┼────────┼──────────────────────────────────┤
│ A surfer dude who relates every  │ A good name for this character   │ 1966   │ Riding the Wave: Life Lessons    │
│ topic to the ocean or surfing.   │ could be "Rip Tide Ryder." This  │        │ from the Ocean                   │
│                                  │ name captures the essence of     │        │                                  │
│                                  │ surfing with "Rip Tide," a       │        │                                  │
│                                  │ common ocean term, and "Ryder,"  │        │                                  │
│                                  │ which evokes the ad

## Posting to the Coop
The [Coop](https://www.expectedparrot.com/explore) is a platform for creating, storing and sharing LLM-based research.
It is fully integrated with EDSL and accessible from your workspace or Coop account page.
Learn more about [creating an account](https://www.expectedparrot.com/login) and [using the Coop](https://docs.expectedparrot.com/en/latest/coop.html).

Here we post the scenarios, survey and results from above, and this notebook:

In [13]:
from edsl import Notebook

In [14]:
n = Notebook(path = "batching_results.ipynb")

In [15]:
n.push(description = "Example code for batching scenarios and combining results", visibility = "public")

{'description': 'Example code for batching scenarios and combining results',
 'object_type': 'notebook',
 'url': 'https://chick.expectedparrot.com/content/e3de929a-cdf5-40b9-aedf-64022cc45fb5',
 'uuid': 'e3de929a-cdf5-40b9-aedf-64022cc45fb5',
 'version': '0.1.33',
 'visibility': 'public'}

To update an object at the Coop:

In [16]:
n = Notebook(path = "batching_results.ipynb")

In [17]:
n.patch(uuid = "e3de929a-cdf5-40b9-aedf-64022cc45fb5", value = n)

{'status': 'success'}